# 基金推薦結果分析

In [1]:
from KNNmodel import *
from rec_funds_offline_cal import *

In [2]:
import numpy as np

In [27]:
con = pypyodbc.connect(
        "DRIVER={SQL Server};SERVER=dbm_public;UID=sa;PWD=01060728;DATABASE=project2017")
data = load_data(con)
train = data['eval_data']['train']
test = data['eval_data']['test']
users_feats_sp = data['users_feats']['sp']
df_purchased = data['purchased_df']


Starting interactions info
Number of rows: 36213
Number of cols: 2074
Sparsity: 0.21%
Ending interactions info
Number of rows: 20607
Number of columns: 2045
Sparsity: 0.32%


In [21]:
idx_to_itemid = data['idx_to_itemid']
idx_to_userid = data['idx_to_userid']
userid_to_idx = data['userid_to_idx']
itemid_to_idx = data['itemid_to_idx']

In [22]:
purchased_ui = data['purchased_ui']

In [23]:
purchased_ui

<20607x2045 sparse matrix of type '<class 'numpy.int32'>'
	with 104793 stored elements in Compressed Sparse Row format>

## 觀察user 
user1 ==> `idx=50` --> `uid = 'A2775267590'`
    
    僅買兩檔基金
    
user2 ==> `idx=100` --> `uid = 'Q1798038310'`
    
    共買61次38檔相異基金
    
user3 ==> `idx=6` ---> `uid = 'A1721889650'`
    
    買6檔相異基金


In [202]:
print('uidx=50,\tuid={}\nuidx=100,\tuid={}\nuidx=3, \tuid={}'
      .format(idx_to_userid[50],idx_to_userid[100],idx_to_userid[3]))


uidx=50,	uid=Q1798038310
uidx=100,	uid=A2775267590
uidx=3, 	uid=A1721889650


In [214]:
df_purchased[df_purchased['身分證字號'] == idx_to_userid[6]]['基金代碼'].unique().shape

(10,)

##### 購買歷史紀錄

In [152]:
user1_purchased = df_purchased[df_purchased['身分證字號'] == idx_to_userid[50]]
user1_purchased

,身分證字號,憑證,基金代碼,商品投資屬性,申購登錄日,扣款次數,申購扣款金額_台幣,國內外基金註記,aum基金型態別,投資型態,投資地區,型態別,aum型態別,aum計算類別,uidxs,iidxs,uidxs_iidxs
57,Q1798038310,016Y5351600135,Y53,RR4,20161108,3,11449.08,1.0,E,a.定時定額,a.境外基金,a.股票型,a.股票型,國外股票型,50,43,50_43
40997,Q1798038310,016465C1700195,465,RR4,20171023,1,302130.00,0.0,E,b.單筆申購,b.國內基金,a.股票型,a.股票型,國內股票型,50,135,50_135


In [203]:
user2_purchased = df_purchased[df_purchased['身分證字號']==idx_to_userid[100]]['基金代碼'].unique()

user2_purchased

array(['58G', 'Y53', '79O', 'MW6', 'Y38', '70H', 'F0F', '74P', 'Y0E',
       'L1G', '389', 'Y31', '259', 'IO6', 'Y82', 'V01', 'L0E', 'IN3',
       '83H', 'X01', 'J15', 'FK6', '366', '63Z', 'J12', 'FL7', '835',
       'IO0', 'J84', 'FK0', 'J0Z', 'F70', 'J0V', '367', '63T', 'Y76',
       '697', 'BMA'], dtype=object)

In [215]:
user3_purchased = df_purchased[df_purchased['身分證字號']==idx_to_userid[6]]['基金代碼'].unique()
user3_purchased

array(['L17', '882', 'V01', 'T34', 'J88', 'Y38', 'T27', 'Y49', '468', 'JA7'], dtype=object)

##### 購買基金種類與AUM級距

In [44]:
data['users_feats']['df'].iloc[50,:]

國內債券型                    0
國內其他型                    0
國內股票型                    1
國內貨幣型                    0
國外債券型                    0
國外其他型                    0
國外股票型                    1
國外貨幣型                    0
aum<10萬                  0
aum10~50萬                0
aum50~100萬               1
aum100~300萬              0
aum>300萬                 0
userid         Q1798038310
Name: 50, dtype: object

In [43]:
data['users_feats']['df'].iloc[100,:]

國內債券型                    1
國內其他型                    1
國內股票型                    9
國內貨幣型                    0
國外債券型                    8
國外其他型                    3
國外股票型                   39
國外貨幣型                    0
aum<10萬                  0
aum10~50萬                0
aum50~100萬               0
aum100~300萬              0
aum>300萬                 1
userid         A2775267590
Name: 100, dtype: object

In [216]:
data['users_feats']['df'].iloc[6,:]

國內債券型                    0
國內其他型                    1
國內股票型                    0
國內貨幣型                    0
國外債券型                    3
國外其他型                    3
國外股票型                    4
國外貨幣型                    0
aum<10萬                  0
aum10~50萬                0
aum50~100萬               0
aum100~300萬              1
aum>300萬                 0
userid         A2524661740
Name: 6, dtype: object

## 建立模型
### UBCF


In [48]:
# model_u = KNNmodel(purchased_ui,kind = 'ubcf')
dt_u,model_u = build_model(purchased_ui,kind ='ubcf',topK=100)

similarity (jaccard) matrix built (ubcf), 
sparsity of similarity: 12.24 %
-- start building topK user array...


100%|███████████████████████████████| 20607/20607 [00:16<00:00, 1226.22users/s]


-- end building topK user array---
start building prediction rating...


100%|████████████████████████████████| 20607/20607 [00:35<00:00, 576.90users/s]


ubcf rating matrix built...

arrange purchased data in rating matrix...
remove=True


100%|███████████████████████████| 2463825/2463825 [00:02<00:00, 1055575.68it/s]


numbers of rows : 20607
numbers of cols: 2045
sparsity of rating: 5.85 %
save into *.rating attribute...
****
time cost for building ubcf (jaccard) model: 0:01:40


* 推薦基金

In [98]:
uids = np.arange(purchased_ui.shape[0])
predall_u = model_u.predict(uids,topN=10)
predall_itemid_u, predall_rating_u = get_itemids_ratings_np(model_u,
                                                            predall_u,idx_to_itemid)

- User1

In [128]:
predall_rating_u[50]

array([ 0.01059757,  0.00756969,  0.00756969,  0.00756969,  0.00756969,
        0.00605575,  0.00605575,  0.00605575,  0.0048446 ,  0.00454292])

In [129]:
predall_itemid_u[50]

array(['X01', 'Y38', 'T34', 'MU7', 'T38', 'UN8', '460', '68C', '423', 'Y13'], dtype=object)

- user2

In [145]:
predall_rating_u[100]

array([ 0.06197649,  0.04211997,  0.04075031,  0.03401511,  0.02814265,
        0.02547057,  0.02430714,  0.02328923,  0.02312483,  0.02280029])

In [104]:
predall_itemid_u[100]

array(['MU7', '770', 'MU5', 'J11', 'J83', 'J1F', 'AE7', 'IQ4', '832', 'L0C'], dtype=object)

- user3

In [225]:
predall_rating_u[6]

array([ 0.00664923,  0.00564468,  0.00533488,  0.00281366,  0.00222435,
        0.00183865,  0.00168323,  0.00160577,  0.00158142,  0.001555  ])

In [224]:
predall_itemid_u[6]

array(['X01', 'Y57', '770', 'MU7', 'Y53', 'AE7', 'MU5', 'L0C', 'T38', '83Y'], dtype=object)

##### 最相似的top K 人

In [155]:
# uid =50
topK = 100
np.argsort(model_u.sim[50,:].A)[0,:-topK-1:-1]

array([   50,  5302, 13850, 15234,  9024, 15149, 11134, 19840, 19197,
       15016,  9052,  9051,  9033,  5433, 14875, 15157,  9015, 19885,
        1629, 14645, 20419, 14603,  5802, 14587,  5890,  5906, 15151,
       19824, 19933,  4562, 20494, 10795,  9393, 18843,  9369, 16186,
       16145, 10836,  4407, 15885, 18856,   125, 20453, 10907, 20470,
       19745, 20466, 15512, 15421, 15390, 15362, 15300, 11981,  5231,
        5907, 18603, 16303, 11311, 20098,   471, 20128, 13171, 19178,
       20146,   426, 12966,  1544, 20174, 20178,  7507, 12798, 11781,
       19026, 12640,  8430, 12587, 12406,  1503, 11931, 12218, 12073,
       20356, 13453,  6875, 19185, 19189,  6011,  8892, 14312, 11357,
       14177,  6214, 14134, 14128, 14123, 14106,  6785,  6272, 14048, 14033], dtype=int64)

In [162]:
np.argsort(model_u.sim[100,:].A)[0,:-topK-1:-1]

array([  100,  4244,  6599,  4493,  1941,   211,  1663,  4050,  4047,
        2145,  3258,  2262,  1321,  2525,   512,  1592,  8633,  2258,
        1664,  4323,  2551,  1802,  3274,  1085,  5239,  1594,   275,
        7710,  3154,  7142,  7531,  3686,  9439,  1906,  8130,  2416,
        1017,  2663,  3829,  5005,  3788,  3076,  2350,  9183,  5543,
       11930,  2917,  2695,  3113, 10698,  1458, 11894,   173,   809,
        2194,  8414,   691,  3312,   970, 10362,  6811,  1819,  1406,
        3040,  2156,  2368,  4238,   169, 12070,  1437,    14,  2310,
       11348,  4185,  2676, 12903,  5958,  6769,  2377,  5378,  3172,
        2794,  2879,  1849,   576,   396,   568,  1067,   537,   547,
        2008,  9156,  7513,  2540,    63,  6476,  9225,  1159,  7652,    84], dtype=int64)

In [226]:
np.argsort(model_u.sim[6,:].A)[0,:-topK-1:-1]

array([    6,  5698,  8911,  2829, 11530,  6928,  1379,  5724, 15297,
       11297,     1,  8617, 13358, 19439,  1901,   115,  1300,  9341,
       13760, 12393, 13980,  3929,  9329, 14087,  3138,  5467, 17941,
       15853,  2652, 17743, 13425,  8862,  3857, 12754, 19226, 20312,
        9433,  6448, 10875, 17889, 16008, 17921, 15513, 18837,  7473,
        5883, 12417,  5935, 10053, 14670,  2449, 12157,  4749, 16637,
       12195, 19756,  5035,  6797, 18862,  1973, 12130,  4887, 19684,
        4396, 10746,  5284, 11721,  9940, 15782,  1655,  1854, 14878,
       17090,  5540,  7597, 12356, 13545,  7277, 15233, 20473, 17297,
        6720,  8434,  6164, 18697,  1193, 11558,  7480,  9833,  6220,
        3446,  4717,  4299, 19234, 12563, 18544, 11321, 19723, 13346, 13912], dtype=int64)

相似度

In [140]:
predall_rating_u[50]

array([ 0.01059757,  0.00756969,  0.00756969,  0.00756969,  0.00756969,
        0.00605575,  0.00605575,  0.00605575,  0.0048446 ,  0.00454292])

In [165]:
predall_rating_u[100]

array([ 0.06197649,  0.04211997,  0.04075031,  0.03401511,  0.02814265,
        0.02547057,  0.02430714,  0.02328923,  0.02312483,  0.02280029])

<1x100 sparse matrix of type '<class 'numpy.float32'>'
	with 73 stored elements in Compressed Sparse Row format>

##### 統計這些人買了什麼?

In [81]:
topK = 100
most_sim_to_user1 = np.argsort(model_u.sim[50,:].A[0])[:-topK-1:-1]

In [84]:
def getUid(x):
    return idx_to_userid.get(x)

In [85]:
most_sim_to_user1_id = list(map(getUid,list(most_sim_to_user1)))

In [175]:
group_user1 = df_purchased[df_purchased['身分證字號'].isin(most_sim_to_user1_id)].groupby(['基金代碼'])['基金代碼'].count()
group_user1.sort_values(ascending=False).head(20)

## 推: 'X01', 'Y38', 'T34', 'MU7', 'T38', 'UN8', '460', '68C', '423', 'Y13'

基金代碼
Y53    90
465    24
X01     7
MU7     5
Y38     5
T34     5
T38     5
460     4
UN8     4
68C     4
55R     3
423     3
Y79     3
770     2
83Y     2
J1F     2
MB0     2
H37     2
FL7     2
N31     2
Name: 基金代碼, dtype: int64

____

# note1:

    - 買的少的用戶，參考的(相似的)用戶也買得少!!
    - 買的多的用戶，參考的用戶也買較多

In [173]:
group_user1.sum() # 100個用戶僅買232次基金

232

In [174]:
group_user2.sum()

4445

In [103]:
most_sim_to_user2 = np.argsort(model_u.sim[100,:].A[0])[:-topK-1:-1]
most_sim_to_user2_id = list(map(getUid,list(most_sim_to_user2)))
group_user2 = df_purchased[df_purchased['身分證字號'].isin(most_sim_to_user2_id)].groupby(['基金代碼'])['基金代碼'].count()
group_user2.sort_values(ascending=False).head(10)

基金代碼
X01    378
Y38    282
J84    170
Y53    168
MU7    126
389     88
MU5     87
63Z     84
FL7     83
770     81
Name: 基金代碼, dtype: int64

In [116]:
## 移除用戶2買
group_user2[~group_user2.isin(user2_purchased)].sort_values(ascending=False).head(20)
## predicted: 
## MU7', '770', 'MU5', 'J11', 'J83', 'J1F', 'AE7', 'IQ4', '832', 'L0C'

基金代碼
X01    378
Y38    282
J84    170
Y53    168
MU7    126
389     88
MU5     87
63Z     84
FL7     83
770     81
V01     67
J0Z     66
J11     66
J83     58
IN3     55
J1F     53
L0C     50
AE7     49
832     49
IQ4     45
Name: 基金代碼, dtype: int64

In [233]:
most_sim_to_user3 = np.argsort(model_u.sim[6,:].A[0])[:-topK-1:-1]
most_sim_to_user3_id = list(map(getUid,list(most_sim_to_user3)))
group_user3 = df_purchased[df_purchased['身分證字號'].isin(most_sim_to_user3_id)].groupby(['基金代碼'])['基金代碼'].count()
# group_user3.sort_values(ascending=False).head(10)
group_user3[~group_user3.isin(user3_purchased)].sort_values(ascending=False).head(10)

基金代碼
T34    88
L17    81
Y38    74
V01    31
X01    13
Y57    13
J88    13
770    13
882    13
Y49     9
Name: 基金代碼, dtype: int64

In [234]:
group_user3.sum()

493

# note2

    - 相似用戶重複買基金會**劇烈**影響結果

In [243]:
group_user1.sort_values(ascending=False).head(20)

基金代碼
Y53    90
465    24
X01     7
MU7     5
Y38     5
T34     5
T38     5
460     4
UN8     4
68C     4
55R     3
423     3
Y79     3
770     2
83Y     2
J1F     2
MB0     2
H37     2
FL7     2
N31     2
Name: 基金代碼, dtype: int64

In [245]:
df_purchased[df_purchased['身分證字號']==idx_to_userid[5320]]
## ## 推: 'X01', 'Y38', 'T34', 'MU7', 'T38', 'UN8', '460', '68C', '423', 'Y13'

,身分證字號,憑證,基金代碼,商品投資屬性,申購登錄日,扣款次數,申購扣款金額_台幣,國內外基金註記,aum基金型態別,投資型態,投資地區,型態別,aum型態別,aum計算類別,uidxs,iidxs,uidxs_iidxs
8388,P2728788800,138X0751600067,X07,RR4,20161024,3,11468.76,1.0,E,a.定時定額,a.境外基金,a.股票型,a.股票型,國外股票型,5320,62,5320_62
135751,P2728788800,138Y7641600270,Y76,RR2,20161021,1,62465.00,1.0,B,b.單筆申購,a.境外基金,b.債券型,b.債券型,國外債券型,5320,1196,5320_1196


In [246]:
predall_itemid_u[50]

array(['X01', 'Y38', 'T34', 'MU7', 'T38', 'UN8', '460', '68C', '423', 'Y13'], dtype=object)

_______

# 舊資料測試

In [260]:
con = pypyodbc.connect(
        "DRIVER={SQL Server};SERVER=dbm_public;UID=sa;PWD=01060728;DATABASE=test")


In [261]:
df_gt2 = pd.read_sql("select * from ihong_基金推薦demo_申購紀錄",con)

In [265]:
purchased_ui, userid_to_idx,\
idx_to_userid, itemid_to_idx,idx_to_itemid = df_to_spmatrix(df_gt2,'身分證字號','基金代碼')


In [268]:
dt_u,model_u = build_model(purchased_ui,kind ='ubcf',topK=100)

similarity (jaccard) matrix built (ubcf), 
sparsity of similarity: 13.44 %
-- start building topK user array...


100%|████████████████████████████████| 26324/26324 [00:28<00:00, 927.31users/s]


-- end building topK user array---
start building prediction rating...


100%|████████████████████████████████| 26324/26324 [00:51<00:00, 510.88users/s]


ubcf rating matrix built...

arrange purchased data in rating matrix...
remove=True


100%|████████████████████████████| 3127487/3127487 [00:03<00:00, 980130.70it/s]


numbers of rows : 26324
numbers of cols: 2149
sparsity of rating: 5.53 %
save into *.rating attribute...
****
time cost for building ubcf (jaccard) model: 0:02:48


# 觀察幾種用戶

1. 成交基金種類少 (user1: `idx = 172`)
    - 相異基金2, 憑證數 2
2. 成交基金種類多多 (user2: `idx=0`)
    - 相異基金42, 憑證數 59
3. 成交基金種類一般 (user3: `idx=10`) 
    - 相異基金10 , 憑證數14

In [298]:

df_gt2[df_gt2['身分證字號']==idx_to_userid[0]]
# user1_purchased = df_purchased[df_purchased['身分證字號'] == idx_to_userid[50]]
# user1_purchased
df_gt2[df_gt2['身分證字號']==idx_to_userid[172]]['基金代碼'].count()

2

In [ ]:
most

In [272]:
uids = np.arange(purchased_ui.shape[0])
predall_u = model_u.predict(uids,topN=10)
predall_itemid_u, predall_rating_u = get_itemids_ratings_np(model_u,
                                                            predall_u,idx_to_itemid)

In [299]:
most_sim_to_user1 = np.argsort(model_u.sim[172,:].A[0])[:-topK-1:-1]
most_sim_to_user2 = np.argsort(model_u.sim[0,:].A[0])[:-topK-1:-1]
most_sim_to_user3 = np.argsort(model_u.sim[10,:].A[0])[:-topK-1:-1]

In [301]:
most_sim_to_user1_id = list(map(getUid,list(most_sim_to_user1)))
most_sim_to_user2_id = list(map(getUid,list(most_sim_to_user2)))
most_sim_to_user3_id = list(map(getUid,list(most_sim_to_user3)))

In [307]:
print('user1 sim group 憑證數',df_gt2[df_gt2['身分證字號'].isin(most_sim_to_user1_id)]['基金代碼'].count())
print('user2 sim group 憑證數',df_gt2[df_gt2['身分證字號'].isin(most_sim_to_user2_id)]['基金代碼'].count())
print('user3 sim group 憑證數',df_gt2[df_gt2['身分證字號'].isin(most_sim_to_user3_id)]['基金代碼'].count())

user1 sim group 憑證數 286
user2 sim group 憑證數 5549
user3 sim group 憑證數 732


In [ ]:
## 買啥??推薦什麼??
user1_purchased = df_gt2[df_gt2['身分證字號']==idx_to_userid[172]]['基金代碼'].unique()
user2_purchased = df_gt2[df_gt2['身分證字號']==idx_to_userid[0]]['基金代碼'].unique()
user3_purchased = df_gt2[df_gt2['身分證字號']==idx_to_userid[10]]['基金代碼'].unique()

In [364]:
group_user1 = df_gt2[df_gt2['身分證字號'].isin(most_sim_to_user1_id)].groupby(['基金代碼'])['基金代碼'].count()
print('user1買買:{}'.format(user1_purchased))
print('====\ngroup 1 愛買\n====\n',group_user1[~group_user1.index.isin(user1_purchased)].sort_values(ascending=False).head(20))
print('====\n推啥\n====\n',predall_itemid_u[172])

user1買買:['63Z' 'FL7']
====
group 1 愛買
====
 基金代碼
UN8    23
J01    10
IQ4     6
J84     5
V01     5
72C     4
363     4
78S     4
T32     4
J99     3
770     3
L50     3
J21     3
J0Z     3
H02     2
776     2
61O     2
L0V     2
MB0     2
J0Y     2
Name: 基金代碼, dtype: int64
====
推啥
====
 ['UN8' 'V01' 'IQ4' '363' 'J01' 'J0Z' 'J84' '770' 'J21' 'L50']


In [368]:
group_user2 = df_gt2[df_gt2['身分證字號'].isin(most_sim_to_user2_id)].groupby(['基金代碼'])['基金代碼'].count()
print('user1買買\n====\n{}'.format(user2_purchased))
print('====\ngroup 2 愛買\n====\n',group_user2[~group_user2.index.isin(user2_purchased)].sort_values(ascending=False).head(20))
print('====\n推啥\n====\n',predall_itemid_u[0])

user1買買
====
['J94' 'F79' '74T' '776' 'T34' '78S' 'J0L' 'MB4' '63Z' 'J0Z' 'V01' '59V'
 '78F' '315' '396' 'X01' '832' 'L0S' '287' 'IQ4' 'J99' 'L03' 'J1F' '369'
 'MB0' 'Y57' 'J88' 'L96' '366' 'JA7' 'Y48' 'F0K' 'Y53' 'L98' '675' '460'
 '768' '865' '371' '83B' 'J87' '676']
====
group 2 愛買
====
 基金代碼
AE7    153
J84    103
Y38    100
UN8     87
GJ8     76
78M     62
J0A     59
MU7     57
83F     54
FL7     52
J21     52
L91     47
J11     47
Y37     45
MU5     44
F0O     41
L02     41
389     39
L0C     33
IN3     32
Name: 基金代碼, dtype: int64
====
推啥
====
 ['AE7' 'J84' '83F' '78M' 'J21' 'GJ8' 'Y38' 'UN8' 'MU7' '770']


In [369]:
group_user3 = df_gt2[df_gt2['身分證字號'].isin(most_sim_to_user3_id)].groupby(['基金代碼'])['基金代碼'].count()
print('user3買買\n====\n{}'.format(user3_purchased))
print('====\ngroup 3\n====\n',group_user3[~group_user3.index.isin(user3_purchased)].sort_values(ascending=False).head(10))
print('====\n推啥\n====\n',predall_itemid_u[10])

user3買買
====
['L16' 'J0A' 'JA7' 'AE7' '66R' 'Y79' '677' 'Y53' '83F' '61R']
====
group 3
====
 基金代碼
MC5    17
UN8    15
J0Z    14
T34    12
J99    12
Y49    12
Y38    11
J77     9
V01     9
676     8
Name: 基金代碼, dtype: int64
====
推啥
====
 ['MC5' 'J0Z' 'UN8' 'J99' 'T34' 'J77' 'Y49' '460' 'GJ8' '78F']
